In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import xgboost as xgb
import pickle
import re
import math

In [2]:
df = pd.read_csv('./Phishing_Dataset.csv')

def checkSpecial(string): 
    # Make own character set and pass  
    # this as argument in compile method 
    regex = re.compile('[@_!#$%^&*()<>?|}{~]') 
    i=0
    for char in string:   
        if regex.search(char): 
            i+=1
    return i

def getNums(str):
    i=0
    for char in str: 
        if char.isdigit():
            i+=1
    return i

def entropy(url):
        string = url.strip()
        prob = [float(string.count(c)) / len(string) for c in dict.fromkeys(list(string))]
        entropy = sum([(p * math.log(p) / math.log(2.0)) for p in prob])
        return entropy

def numSubDomains(url):
    subdomains = url.split('http')[-1].split('//')[-1].split('/')
    return len(subdomains)-1

def feature_transform(df):
    df.insert(2,'len_url', [len(url) for url in df['URL']])


    df.insert(2,'numerical',[getNums(url) for url in df['URL']])

    df.insert(2,'special',[checkSpecial(url) for url in df['URL']])

    df.insert(2, 'hasPercent', [('%' in url) for url in df['URL']])

    #df.insert(2, 'entropy', [entropy(url) for url in df['URL']])

    df.insert(2, 'numSD', [numSubDomains(url) for url in df['URL']])

    del df['URL']

feature_transform(df)

#split into parameters and label for supervised learning
X, y = df.iloc[:,:-1], df.iloc[:, -1]
data_dmatrix = xgb.DMatrix(data=X, label=y)



In [3]:
#split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
#train model
xg_reg = xgb.XGBClassifier(learning_rate=0.2)

xg_reg.fit(X_train,y_train)




XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.2, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [4]:
#predictions /validation 
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score
preds = xg_reg.predict(X_test)
predictions = [round(value) for value in preds]
rmse = np.sqrt(mean_squared_error(y_test, predictions))
#("RMSE: %f" % (rmse))

#score = accuracy_score(y_test,predictions)
#print(score)
print("F1 Score on training dataset:")
f1 = f1_score(y_test, predictions)
print(f1)

# testing 
# test_data = pd.read_csv('./Phishing_Test.csv')
# ids = test_data['Unnamed: 0']
# urls = test_data['URL']
# del test_data['Unnamed: 0']
# del test_data['Label']

# feature_transform(test_data)
# labels = xg_reg.predict(test_data)

# def unfeaturize(data):
#     del data['numSD']
#     del data['hasPercent']
#     del data['special']
#     del data['numerical']
#     del data['len_url']

# unfeaturize(test_data)
# test_data.insert(0, '',ids )
# test_data.insert(4, 'URL', urls)
# test_data.insert(5, 'Label', labels)

# test_data.to_csv(r'./Phishing_Labels.csv', index=False)


F1 Score on training dataset:
0.9408450704225352
